In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import re
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import requests
from datetime import datetime
import time

In [2]:
url = "https://www.transfermarkt.com/joshua-kimmich/marktwertverlauf/spieler/161056"

In [3]:
path_to_extension = r'C:\Users\Mirko\AppData\Local\Google\Chrome\User Data\Default\Extensions\cjpalhdlnbpafiamejdnhcphjbkeiagm\1.61.0_0'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)


driver = webdriver.Chrome(options=chrome_options)

In [9]:
driver.get(url)

In [10]:
iframe = driver.find_element(By.ID, "sp_message_iframe_953358")
driver.switch_to.frame(iframe)
driver.find_element(By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button").click()

In [11]:
driver.switch_to.parent_frame()

In [328]:
result_list = []

In [329]:
values_raw = driver.find_element(By.CLASS_NAME,"box").find_element(By.CLASS_NAME, "chart-dots").find_elements(By.CSS_SELECTOR, "*")

In [ ]:
values_raw

In [ ]:
element_to_hover_over = values_raw[1]
hover = ActionChains(driver).move_to_element(element_to_hover_over)
hover.perform()

In [ ]:
result = driver.find_element(By.CLASS_NAME, "chart-tooltip.svelte-xxbpuy").find_element(By.CLASS_NAME, "left.svelte-xxbpuy").text

In [334]:
result

'Jan 24, 2014\nMarket value: €500k\nClub: RB Leipzig\nAge: 18'

In [335]:
date = re.search("^.*\n", result).group(0)
date = re.sub("\n", "", date)	

value = re.search("Market value:.*\n", result).group(0)
value = re.sub("\n", "", value)	
value = re.sub(".*: ","", value)
value = re.sub("k","000", value)
value = re.search("\\d+", value).group(0)

club = re.search("Club:.*\n", result).group(0)
club = re.sub("\n", "", club)	
club = re.sub(".*: ","", club)

age = re.search("Age:.*", result).group(0)
age = re.sub("\n", "", age)	
age = re.sub(".*: ","", age)
age

'18'

In [ ]:
result_list.append({"Date": date, "Value": value, "Club": club, "Age": age})

In [337]:
result_list

[{'Date': 'Jan 24, 2014',
  'Value': '500000',
  'Club': 'RB Leipzig',
  'Age': '18'}]

In [23]:
year = 2024
url = f"https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id={year}"
"https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2024"
"https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2024"
"https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2024"
"https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2024"

'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2024'

In [24]:
path_to_extension = r'C:\Users\Mirko\AppData\Local\Google\Chrome\User Data\Default\Extensions\cjpalhdlnbpafiamejdnhcphjbkeiagm\1.61.0_0'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)


driver = webdriver.Chrome(options=chrome_options)

In [25]:
driver.get(url)

In [26]:
iframe = driver.find_element(By.ID, "sp_message_iframe_953358")
driver.switch_to.frame(iframe)
driver.find_element(By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button").click()

In [27]:
html = driver.page_source

In [28]:
soup = BeautifulSoup(html, "html")

In [29]:
html_table = soup.find(id = "yw1").find("table")
table = pd.read_html(StringIO(str(html_table)))[0]
club_name = table.loc[table["Club.1"].notnull(),"Club.1"]
links_raw = soup.find(id = "yw1").find("tbody").find_all(class_ = "zentriert no-border-rechts")
links = []
for i in links_raw:
    links.append(f"https://www.transfermarkt.com{i.find("a")["href"]}")
final_table = pd.DataFrame({"Club": club_name, "Link": links, "Season": f"{year}/{year+1}"})
final_table

,Club,Link,Season
0,Bayern Munich,https://www.transfermarkt.com/fc-bayern-munche...,2024/2025
1,Bayer 04 Leverkusen,https://www.transfermarkt.com/bayer-04-leverku...,2024/2025
2,RB Leipzig,https://www.transfermarkt.com/rasenballsport-l...,2024/2025
3,Borussia Dortmund,https://www.transfermarkt.com/borussia-dortmun...,2024/2025
4,VfB Stuttgart,https://www.transfermarkt.com/vfb-stuttgart/st...,2024/2025
5,Eintracht Frankfurt,https://www.transfermarkt.com/eintracht-frankf...,2024/2025
6,VfL Wolfsburg,https://www.transfermarkt.com/vfl-wolfsburg/st...,2024/2025
7,SC Freiburg,https://www.transfermarkt.com/sc-freiburg/star...,2024/2025
8,TSG 1899 Hoffenheim,https://www.transfermarkt.com/tsg-1899-hoffenh...,2024/2025
9,Borussia Mönchengladbach,https://www.transfermarkt.com/borussia-monchen...,2024/2025


In [30]:
driver.get(links[0])

In [31]:
html = driver.page_source
soup = BeautifulSoup(html, "html")

player_links_raw = soup.find(id = "yw1").find("tbody").find_all(class_ = "rechts hauptlink")
player_links = []
for i in player_links_raw:
    player_links.append(f"https://www.transfermarkt.com{i.find("a")["href"]}")
player_links

['https://www.transfermarkt.com/manuel-neuer/marktwertverlauf/spieler/17259',
 'https://www.transfermarkt.com/daniel-peretz/marktwertverlauf/spieler/468539',
 'https://www.transfermarkt.com/sven-ulreich/marktwertverlauf/spieler/40680',
 'https://www.transfermarkt.com/dayot-upamecano/marktwertverlauf/spieler/344695',
 'https://www.transfermarkt.com/min-jae-kim/marktwertverlauf/spieler/503482',
 'https://www.transfermarkt.com/hiroki-ito/marktwertverlauf/spieler/353892',
 'https://www.transfermarkt.com/eric-dier/marktwertverlauf/spieler/175722',
 'https://www.transfermarkt.com/tarek-buchmann/marktwertverlauf/spieler/744729',
 'https://www.transfermarkt.com/alphonso-davies/marktwertverlauf/spieler/424204',
 'https://www.transfermarkt.com/raphael-guerreiro/marktwertverlauf/spieler/170986',
 'https://www.transfermarkt.com/adam-aznou/marktwertverlauf/spieler/938146',
 'https://www.transfermarkt.com/josip-stanisic/marktwertverlauf/spieler/483046',
 'https://www.transfermarkt.com/sacha-boey/mar

In [32]:
driver.get(player_links[0])

In [157]:
def get_player_values(driver = driver):
    finished = False
    tries = 0
    while (finished == False & tries <= 10):
        try:
            values_raw = driver.find_element(By.CLASS_NAME,"box").find_element(By.CLASS_NAME, "chart-dots").find_elements(By.CSS_SELECTOR, "*")
            finished = True
        except:
            finished = False
            tries += 1
            if (tries < 10):
                print(f"try # {tries} did noot work. i will try again. stop after 10 tries")
            else:
                print("i tried 10 times. i will stop now")
                break
    result_list = []
    for i in values_raw:
        ActionChains(driver).move_to_element(i).perform()
        if i == values_raw[len(values_raw)-1]:
            ActionChains(driver).move_by_offset(-2, 0).perform() # a little tweak because the last element is not recognized	
        finished = False
        tries = 0
        while (finished == False & tries <= 10):
            try:	
                result = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tm-main"]/div[2]/div[1]/div/tm-market-value-development-graph-extended/div/div/div/p')))
                finished = True
            except:
                finished = False
                tries += 1
                if (tries < 10):
                    print(f"try # {tries} did noot work. i will try again. stop after 10 tries")
                else:
                    print("i tried 10 times. i will stop now")
                    break
        result = result.text
        date = re.search("^.*\n", result).group(0)
        date = re.sub("\n", "", date)	
        
        value = re.search("Market value:.*\n", result).group(0)
        value = re.sub("\n", "", value)
        value = re.sub("\\.\\d\\dm","000000", value)
        value = re.sub(".*: ","", value)
        value = re.sub("k","000", value)
        value = re.search("\\d+", value).group(0)
        
        club = re.search("Club:.*\n", result).group(0)
        club = re.sub("\n", "", club)
        club = re.sub(".*: ","", club)
        
        age = re.search("Age:.*", result).group(0)
        age = re.sub("\n", "", age)
        age = re.sub(".*: ","", age)
        
        result_list.append({"Date": date, "Value": value, "Club": club, "Age": age})
    
    html = driver.page_source 
    soup = BeautifulSoup(html, "html")
    
    name = soup.find(class_ = 'data-header').find("h1").text
    name = re.search("\n.*$", name).group(0)
    name = re.sub("\n","", name)
    name.lstrip().rstrip()
    
    birthdate = soup.find(class_ = 'data-header__details').find('ul').find("li").text 
    birthdate = re.search("\n.*\\(", birthdate).group(0)
    birthdate = re.sub("(\n)|(\\()","", birthdate)
    birthdate.lstrip().rstrip()
    
    result_df = pd.DataFrame(result_list)
    result_df["player_name"] = name
    result_df["player_birthdate"] = birthdate
    return result_df

In [158]:
all_players = pd.DataFrame()

for i in player_links:
    driver.get(i)
    one_player = get_player_values(driver)
    player_id = re.sub(".*spieler/","",i)
    one_player["player_id"] = player_id
    print(f"Got player: {one_player.loc[1,"player_name"].strip()}")
    all_players = pd.concat([all_players, one_player], ignore_index = True)


Got player: Manuel Neuer
Got player: Daniel Peretz
try # 1 did noot work. i will try again. stop after 10 tries
Got player: Sven Ulreich
Got player: Dayot Upamecano
Got player: Min-jae Kim
Got player: Hiroki Ito
Got player: Eric Dier
Got player: Tarek Buchmann
Got player: Alphonso Davies
Got player: Raphaël Guerreiro
Got player: Adam Aznou
try # 1 did noot work. i will try again. stop after 10 tries
Got player: Josip Stanisic
try # 1 did noot work. i will try again. stop after 10 tries
Got player: Sacha Boey
Got player: Joshua Kimmich
Got player: João Palhinha
Got player: Aleksandar Pavlovic
Got player: Konrad Laimer
Got player: Leon Goretzka
Got player: Jamal Musiala
Got player: Serge Gnabry
Got player: Kingsley Coman
try # 1 did noot work. i will try again. stop after 10 tries
Got player: Mathys Tel
Got player: Arijon Ibrahimovic
Got player: Michael Olise
Got player: Leroy Sané
Got player: Nestory Irankunda
try # 1 did noot work. i will try again. stop after 10 tries
Got player: Thom